# Assignment 7

## Submit as an HTML file

### Print your name below

In [30]:
print("Malia Yap")

Malia Yap


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [3]:
# Write your answer here:

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [48]:
results = pd.read_csv("/Users/maliayap/Desktop/qtm151spring2025/Assignments/assignment7/data/results.csv")

### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [55]:
# Write your answer here
print(results.dtypes)

list_old = ['\\N']
list_new = [np.nan]

results["milliseconds"] = results["milliseconds"].replace(list_old, list_new)
results["race_time_ms"] = pd.to_numeric(results["milliseconds"])


resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
race_time_ms       float64
dtype: object


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [31]:
list_old = ['\\N']
list_new = [np.nan]

results["position"] = results["position"].replace(list_old, list_new)
results["position"] = pd.to_numeric(results["position"])

bins_x = [1,3, 10, 20, 500]
labels_x = ["Podium",
            "Points",
            "Midfield",
            "Backmarker",]

results["finish_category"] = pd.cut(results["position"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [34]:
# Write your answer here
results["milliseconds"] = pd.to_numeric(results["milliseconds"])
results["race_duration_minutes"] = results["milliseconds"] / (1000 * 60)

avg_race_duration = results.groupby("constructorId")["race_duration_minutes"].mean()
print(avg_race_duration[0:5])

constructorId
1     97.461541
2     94.119229
3     96.770410
4     95.154676
5    102.491413
Name: race_duration_minutes, dtype: float64


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [41]:
# Write your answer here


results_agg = results.groupby("driverId").agg(
                            mean_position = ('position','mean'),
                          total_points = ('points','sum'),
                          number_races = ("number", "size"),
                          best_position = ("position", "min")
)

display(results_agg)

sorted_driver_stats = results_agg.sort_values(by='total_points', ascending=False)
top10 = sorted_driver_stats.head(10)
print(top10)


,mean_position,total_points,number_races,best_position
driverId,,,,
1,3.383803,4396.5,310,1.0
2,8.355072,259.0,184,2.0
3,6.355932,1594.5,206,1.0
4,6.257732,2061.0,358,1.0
5,11.835165,105.0,112,1.0
...,...,...,...,...
852,12.352941,44.0,44,4.0
853,17.812500,0.0,22,14.0
854,15.052632,12.0,44,6.0


          mean_position  total_points  number_races  best_position
driverId                                                          
1              3.383803        4396.5           310            1.0
20             5.282443        3098.0           300            1.0
4              6.257732        2061.0           358            1.0
830            4.022388        1983.5           163            1.0
8              6.238596        1873.0           352            1.0
822            6.296089        1778.0           201            1.0
3              6.355932        1594.5           206            1.0
30             3.701245        1566.0           308            1.0
817            8.261538        1307.0           232            1.0
18             7.409091        1235.0           309            1.0


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [53]:
# Write your answer here

results["grid"] = pd.to_numeric(results["grid"])
results['laps'] = pd.to_numeric(results["laps"])
results = results.dropna()


model = smf.ols('points ~ grid + laps', data = results)
results_model = model.fit()

print(results_model.summary())

predicted_points = results_model.predict({'grid': [3], 'laps': [55]})

print(predicted_points)



                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        13:26:37   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0